In [9]:
import pandas as pd
import sqlite3

# SQLite database file path
sqlite_file_path = '/home/aj/Desktop/data/pdf_annotator_app.db'

# CSV file path
csv_file_path = '/home/aj/Desktop/data/files_list.csv'

# Text file containing the list
list_file_path = '/home/aj/Desktop/data/fields_list.lst'

# Read CSV file using pandas
df_csv = pd.read_csv(csv_file_path)

# Read the list from the file
with open(list_file_path, 'r') as file:
    field_list = [line.strip() for line in file]

# Connect to SQLite database
conn = sqlite3.connect(sqlite_file_path)
cursor = conn.cursor()

# Define table creation query
table_name = 'files_master_table'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    filename TEXT,
    filepath TEXT,
    extracted BOOLEAN
)
"""

# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_data_query = f"INSERT INTO {table_name} (filename, filepath, extracted) VALUES (?, ?, ?)"

for _, row in df_csv.iterrows():
    data_tuple = (row['filename'], row['filepath'], row['extracted'])
    cursor.execute(insert_data_query, data_tuple)

conn.commit()

# Define table creation query
table_name = 'fields_master_table'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fields TEXT
)
"""
# Execute the table creation query
cursor.execute(create_table_query)
conn.commit()

# Insert data into the table
insert_data_query = f"INSERT INTO {table_name} (fields) VALUES (?)"

for field in field_list:
    data_tuple = (field,)
    cursor.execute(insert_data_query, data_tuple)

conn.commit()


# Close the database connection
cursor.close()
conn.close()

print(f"Database and table created successfully from {csv_file_path}.")


Database and table created successfully from /home/aj/Desktop/data/files_list.csv.


In [8]:
import os

# SQLite database file path
sqlite_file_path = '/home/aj/Desktop/data/pdf_annotator_app.db'

# Check if the file exists before attempting to delete
if os.path.exists(sqlite_file_path):
    # Close the database connection if it's open
    # This is important to avoid potential issues when deleting the file
    try:
        conn.close()
    except NameError:
        pass

    # Delete the SQLite database file
    os.remove(sqlite_file_path)
    print(f"The SQLite database file '{sqlite_file_path}' has been deleted.")
else:
    print(f"The SQLite database file '{sqlite_file_path}' does not exist.")

The SQLite database file '/home/aj/Desktop/data/pdf_annotator_app.db' has been deleted.


In [ ]:
import 